In [1]:
import statsmodels.api as sm 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action="ignore")
import plotly.express as px
import plotly.graph_objects as go

In [2]:
df_rolled = pd.read_parquet("../data/output/df_model.parquet")
df_rolled.shape

(125776, 31)

In [3]:
df_rolled['PPG'].nunique()

314

In [4]:
print("Number of unique retailer PPG combinations",df_rolled[['PPG','Retailer_Name']].drop_duplicates().shape[0])

Number of unique retailer PPG combinations 743


In [5]:
df_rolled = df_rolled.loc[df_rolled['No_Total_Sales_Flag']==0]
print(df_rolled.shape)

(14709, 31)


In [6]:
print("Number of unique retailer PPG combinations",df_rolled[['PPG','Retailer_Name']].drop_duplicates().shape[0])

Number of unique retailer PPG combinations 465


In [7]:
model_levels = ["PPG","Retailer_Name"] 

In [8]:
#Log of price and sales to  build log-log model
df_rolled['log_sales'] = np.log(df_rolled['Total_Volume'])
df_rolled['log_price'] = np.log(df_rolled['Avg_Price_Per_KG'])

In [9]:
var_dep = 'log_sales'
vars_ind = ['log_price','Distribution_wtd','Category_Seasonality','Category_Trend']
vars_all = [var_dep] + vars_ind
print(vars_all)

df_rolled[vars_all].isnull().sum()

['log_sales', 'log_price', 'Distribution_wtd', 'Category_Seasonality', 'Category_Trend']


log_sales               0
log_price               0
Distribution_wtd        1
Category_Seasonality    0
Category_Trend          0
dtype: int64

In [10]:
#Distribution can be replaced with mean for now, should be okay more or less
df_rolled['Distribution_numeric'] = df_rolled['Distribution_numeric'].fillna(df_rolled.groupby(model_levels)['Distribution_numeric'].transform('mean'))
df_rolled['Distribution_wtd'] = df_rolled['Distribution_wtd'].fillna(df_rolled.groupby(model_levels)['Distribution_wtd'].transform('mean'))

In [11]:
df_rolled = df_rolled.loc[~df_rolled['Distribution_numeric'].isna()]
df_rolled = df_rolled.loc[~df_rolled['Distribution_wtd'].isna()]
df_rolled.shape

(14709, 33)

In [12]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

results_summary_coeffs = df_rolled.groupby(model_levels,as_index=False).apply(regress, var_dep, vars_ind)
results_summary_coeffs.head()

,PPG,Retailer_Name,log_price,Distribution_wtd,Category_Seasonality,Category_Trend,intercept
0,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer A,-0.852109,0.123530,0.197584,-6.403994,8.002612
1,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer B,-5.057595,0.007523,0.007206,0.015822,9.290431
2,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer C,-1.630746,0.075442,-0.033026,0.032400,5.522694
3,BRAND A SUBBRAND A.2 ADULT ZAK GRONT_RND 1400_G,Retailer A,-0.302161,0.047439,0.044996,-0.024606,4.752580
4,BRAND A SUBBRAND A.2 ADULT ZAK GRONT_RND 1400_G,Retailer B,-2.076912,0.023631,0.086669,-0.062266,6.300813


In [13]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.rsquared

results_summary_score = df_rolled.groupby(model_levels,as_index=False).apply(regress, var_dep, vars_ind)
results_summary_score.columns = ['PPG','Retailer_Name','RSquared']
results_summary_score.RSquared = results_summary_score.RSquared*100 
results_summary_score.head()

,PPG,Retailer_Name,RSquared
0,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer A,95.718866
1,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer B,83.103341
2,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer C,85.564247
3,BRAND A SUBBRAND A.2 ADULT ZAK GRONT_RND 1400_G,Retailer A,90.628859
4,BRAND A SUBBRAND A.2 ADULT ZAK GRONT_RND 1400_G,Retailer B,96.522362


In [14]:
def calculate_mape(pred,actual):    
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [15]:
results_summary_mape = pd.DataFrame(columns=['PPG_Retailer_Combo','MAPE'])

df_rolled['PPG_Retailer_Combo'] = df_rolled['PPG'] +  "*" + df_rolled['Retailer_Name']

for each_combo in df_rolled['PPG_Retailer_Combo'].unique().tolist():
    curr_df = df_rolled.loc[df_rolled['PPG_Retailer_Combo']==each_combo]
    Y = curr_df[var_dep]
    X = curr_df[vars_ind]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    curr_df['predicted'] = result.get_prediction(X).summary_frame()['mean']
    curr_mape = calculate_mape(curr_df['predicted'] ,curr_df[var_dep])
    results_summary_mape.loc[len(results_summary_mape)] = [each_combo,curr_mape]
    # print(curr_mape)


In [16]:
results_summary_mape[['PPG','Retailer_Name']] = results_summary_mape['PPG_Retailer_Combo'].str.split("*",expand=True)
del results_summary_mape['PPG_Retailer_Combo']
results_summary_mape.head()

,MAPE,PPG,Retailer_Name
0,0.992367,BRAND D SUBBRAND D.2 ADULT POUCH RND_KP_LM_KN ...,Retailer C
1,1.182068,BRAND D SUBBRAND D.2 ADULT POUCH RND_KP_TN_ZLM...,Retailer C
2,1.840325,BRAND D SUBBRAND D.2 ADULT POUCH TON_ZLM_KB_SC...,Retailer B
3,1.299402,BRAND D SUBBRAND D.2 ADULT POUCH TON_ZLM_KB_SC...,Retailer C
4,0.587125,BRAND D SUBBRAND D.2 KITTEN POUCH KIP_RND_TN_Z...,Retailer C


In [17]:
results_combined = pd.merge(results_summary_coeffs,results_summary_score)
results_combined = pd.merge(results_combined,results_summary_mape)
results_combined.head()

,PPG,Retailer_Name,log_price,Distribution_wtd,Category_Seasonality,Category_Trend,intercept,RSquared,MAPE
0,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer A,-0.852109,0.123530,0.197584,-6.403994,8.002612,95.718866,3.210109
1,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer B,-5.057595,0.007523,0.007206,0.015822,9.290431,83.103341,1.809377
2,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer C,-1.630746,0.075442,-0.033026,0.032400,5.522694,85.564247,1.014445
3,BRAND A SUBBRAND A.2 ADULT ZAK GRONT_RND 1400_G,Retailer A,-0.302161,0.047439,0.044996,-0.024606,4.752580,90.628859,3.045934
4,BRAND A SUBBRAND A.2 ADULT ZAK GRONT_RND 1400_G,Retailer B,-2.076912,0.023631,0.086669,-0.062266,6.300813,96.522362,1.311717


In [18]:
results_combined_orig = results_combined.copy(deep=True)
print(results_combined.shape)
results_combined = results_combined.loc[(results_combined['log_price']<=0)&(results_combined['log_price']>=-10)]
results_combined = results_combined.loc[results_combined['RSquared']<=100]
results_combined = results_combined.loc[results_combined['MAPE']<=100]
print(results_combined.shape)

(465, 9)
(343, 9)


In [19]:
fig = px.scatter(results_combined,y='RSquared',x='MAPE',color='log_price',width=800,height=800,hover_data=['PPG','Retailer_Name'])
fig.show()

In [20]:
results_combined_orig.columns

Index(['PPG', 'Retailer_Name', 'log_price', 'Distribution_wtd',
       'Category_Seasonality', 'Category_Trend', 'intercept', 'RSquared',
       'MAPE'],
      dtype='object')

In [21]:
results_combined_orig.head()
for each_col in ['log_price', 'Distribution_wtd',
        'Category_Seasonality', 'Category_Trend',
       'intercept']:
    results_combined_orig.rename(columns={each_col:'coef_'+each_col},inplace=True)
results_combined_orig.head()

,PPG,Retailer_Name,coef_log_price,coef_Distribution_wtd,coef_Category_Seasonality,coef_Category_Trend,coef_intercept,RSquared,MAPE
0,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer A,-0.852109,0.123530,0.197584,-6.403994,8.002612,95.718866,3.210109
1,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer B,-5.057595,0.007523,0.007206,0.015822,9.290431,83.103341,1.809377
2,BRAND A SUBBRAND A.1 ADULT ZAK GROENTE_KIP_RIJ...,Retailer C,-1.630746,0.075442,-0.033026,0.032400,5.522694,85.564247,1.014445
3,BRAND A SUBBRAND A.2 ADULT ZAK GRONT_RND 1400_G,Retailer A,-0.302161,0.047439,0.044996,-0.024606,4.752580,90.628859,3.045934
4,BRAND A SUBBRAND A.2 ADULT ZAK GRONT_RND 1400_G,Retailer B,-2.076912,0.023631,0.086669,-0.062266,6.300813,96.522362,1.311717


In [22]:
results_combined_orig.to_clipboard()

In [23]:
df_waterfall_calc = df_rolled.merge(results_combined_orig,how='left',on=['Retailer_Name','PPG'])
print(df_waterfall_calc.shape)

(14709, 41)


In [24]:
df_waterfall_calc['contri_price'] = np.exp(df_waterfall_calc['log_price']*df_waterfall_calc['coef_log_price'])
df_waterfall_calc['contri_Distribution_wtd'] = df_waterfall_calc['Distribution_wtd']*df_waterfall_calc['coef_Distribution_wtd']
df_waterfall_calc['contri_Category_Seasonality'] = df_waterfall_calc['Category_Seasonality']*df_waterfall_calc['coef_Category_Seasonality']
df_waterfall_calc['contri_Category_Trend'] = df_waterfall_calc['Category_Trend']*df_waterfall_calc['coef_Category_Trend']
df_waterfall_calc['contri_intercept'] = df_waterfall_calc['coef_intercept']

In [25]:
df_waterfall_calc['predicted_sales'] = df_waterfall_calc['contri_price'] \
    + df_waterfall_calc['contri_Distribution_wtd'] \
        +df_waterfall_calc['contri_Category_Seasonality'] \
        + df_waterfall_calc['contri_Category_Trend']\
        + df_waterfall_calc['contri_intercept']
        

In [26]:
df_waterfall_calc['perc_contri_price'] = np.round(((df_waterfall_calc['contri_price']/df_waterfall_calc['predicted_sales'])*100),2)
df_waterfall_calc['perc_contri_Distribution_wtd'] = np.round(((df_waterfall_calc['contri_Distribution_wtd']/df_waterfall_calc['predicted_sales'])*100),2)
df_waterfall_calc['perc_contri_Category_Seasonality'] = np.round(((df_waterfall_calc['contri_Category_Seasonality']/df_waterfall_calc['predicted_sales'])*100),2)
df_waterfall_calc['perc_contri_Category_Trend'] = np.round(((df_waterfall_calc['contri_Category_Trend']/df_waterfall_calc['predicted_sales'])*100),2)
df_waterfall_calc['perc_contri_Baseline'] = np.round(((df_waterfall_calc['contri_intercept']/df_waterfall_calc['predicted_sales'])*100),2)


In [27]:
df_waterfall_calc[0:5].to_clipboard()

In [28]:
df_waterfall_calc.to_clipboard()

In [29]:
df_waterfall_calc[['Total_Volume','predicted_sales','contri_price','perc_contri_price','perc_contri_Distribution_wtd','perc_contri_Baseline']]

,Total_Volume,predicted_sales,contri_price,perc_contri_price,perc_contri_Distribution_wtd,perc_contri_Baseline
0,886.8,705.643162,705.412969,99.97,0.30,-0.25
1,709.6,9.272542,0.073211,0.79,7.52,92.77
2,286.8,7.683096,0.130099,1.69,29.77,68.68
3,769.2,8.564755,0.189589,2.21,20.04,78.49
4,326.4,8.394697,4.041321,48.14,17.00,36.76
...,...,...,...,...,...,...
14704,174.8,8.483939,4.986934,58.78,86.91,-44.82
14705,129.7,8.421067,0.028690,0.34,34.61,64.55
14706,85.4,5.370690,0.877811,16.34,46.22,38.13
14707,160.8,7.970776,0.056032,0.70,29.54,68.96


In [30]:
print(df_waterfall_calc.shape)
filtered_waterfall = df_waterfall_calc.loc[df_waterfall_calc['RSquared']>90]
filtered_waterfall = filtered_waterfall.loc[filtered_waterfall['MAPE']<5]
# filtered_waterfall = filtered_waterfall.loc[(filtered_waterfall['log_price']<=0)&(filtered_waterfall['log_price']>=-3)]
filtered_waterfall = filtered_waterfall.loc[(filtered_waterfall['perc_contri_price']<=55)&(filtered_waterfall['perc_contri_price']>=30)]


filtered_waterfall.shape

(14709, 52)


(131, 52)

In [31]:
filtered_waterfall.groupby(['PPG','Retailer_Name'],as_index=False).agg({'perc_contri_price':'mean'})['PPG'].unique()

array(['BRAND D SUBBRAND D.1.3 KITTEN PAK GRANN_KP_MLK 1000_G',
       'BRAND D SUBBRAND D.2 KITTEN POUCH KIP_RND_TN_ZLM 1200_G_12_ST',
       'BRAND D SUBBRAND D.3 ADULT BLIK EEND_GVGLT 400_G',
       'BRAND D SUBBRAND D.3 ADULT BLIK FORL_ZLM 400_G',
       'BRAND D SUBBRAND D.3 ADULT BLIK LEVR_RND 400_G',
       'BRAND D SUBBRAND D.3 ADULT BLIK TONJN_ZLM 400_G',
       'BRAND D SUBBRAND D.3 ADULT POUCH EEND_LM_KLK_LV_KP_NR_KN_GV 1200_G_12_ST',
       'BRAND F SUBBRAND F.4 ADULT POUCH KIP_RND_KLK_LM_KLF_ND_KN_WLD 340_G_4_ST',
       'BRAND H SUBBRAND H.1 ADULT ZAK GRONT_RND 3000_G',
       'BRAND H SUBBRAND H.2 ADULT BLIK GRONT_LM_PST 400_G',
       'BRAND H SUBBRAND H.4 ADULT POUCH KIP_RND_LV_KLK_LM 1200_G_12_ST',
       'BRAND I SUBBRAND I.3 ADULT ZAK KIP 1400_G',
       'BRAND M SUBBRAND M.1 ADULT ALU ZALM 85_G',
       'BRAND M SUBBRAND M.4 ADULT POUCH ZLM_WTV_KP_RND 1020_G_12_ST'],
      dtype=object)

In [32]:
filtered_waterfall.groupby(['PPG','Retailer_Name'],as_index=False).agg({'perc_contri_price':'mean'})

,PPG,Retailer_Name,perc_contri_price
0,BRAND D SUBBRAND D.1.3 KITTEN PAK GRANN_KP_MLK...,Retailer C,32.080000
1,BRAND D SUBBRAND D.2 KITTEN POUCH KIP_RND_TN_Z...,Retailer C,43.331071
2,BRAND D SUBBRAND D.3 ADULT BLIK EEND_GVGLT 400_G,Retailer A,33.455000
3,BRAND D SUBBRAND D.3 ADULT BLIK FORL_ZLM 400_G,Retailer A,40.047727
4,BRAND D SUBBRAND D.3 ADULT BLIK LEVR_RND 400_G,Retailer C,32.260000
5,BRAND D SUBBRAND D.3 ADULT BLIK TONJN_ZLM 400_G,Retailer A,49.650000
6,BRAND D SUBBRAND D.3 ADULT POUCH EEND_LM_KLK_L...,Retailer C,47.859048
7,BRAND F SUBBRAND F.4 ADULT POUCH KIP_RND_KLK_L...,Retailer C,53.745714
8,BRAND H SUBBRAND H.1 ADULT ZAK GRONT_RND 3000_G,Retailer A,32.111667
9,BRAND H SUBBRAND H.2 ADULT BLIK GRONT_LM_PST 4...,Retailer B,40.850000


In [33]:
# PPG = 'BRAND B SUBBRAND B.1 ADULT POUCH RND_RD_KP_WRT_LM 1200_G_12_ST'
# RETAILER = 'Retailer C'
# col_list = 'perc_contri_Baseline', 'perc_contri_price','perc_contri_Distribution_wtd',\
#     'perc_contri_Category_Seasonality','perc_contri_Category_Trend','predicted_sales'
# value_list = list()
# for each_col in col_list:
#     val = filtered_waterfall.loc[(filtered_waterfall['PPG']==PPG)&(filtered_waterfall['Retailer_Name']==RETAILER),each_col].iloc[0]
#     value_list.append(val)

IndexError: single positional indexer is out-of-bounds

In [ ]:

# fig = go.Figure(go.Waterfall(
#     name = "%Contribution to Sales Volume", orientation = "v",
#     measure = ["absolute", "relative", "relative", "relative", "relative", "total"],
#     x = ["Baseline", "Price", "Distribution", "Category Seasonality", "Category Trend", "Final Volume"],
#     textposition = "outside",
#     text =  [f'{i*1:.2f}%' for i in value_list],
#     y = value_list,
#     connector = {"line":{"color":"rgb(63, 63, 63)"}},
# ))

# fig.update_layout(
#         title = "PPG || " + PPG + "||" + RETAILER,
#         showlegend = True,height=600
# )

# fig.show()

In [ ]:
# import plotly.io as pio
# pio.write_image(fig, '../plots/waterfall.jpg',height=600,width=1200,)